In [21]:
setwd("~/popgenomics/data/fst_hmm")

In [22]:
# library to fit Hidden Markov Models
library(HiddenMarkov)

# library to speed up loading of big tables
library(data.table)

# the library of popgen functions
library(popgenfunctions)

In [23]:
#rnormc, dnormc, pnormc and qnormc are used further on in the ‘dthmm’ function that uses
#the normc distribution from mstep.normc
rnormc <- rnorm #random number generator
dnormc <- dnorm #density
pnormc <- pnorm #distribution function
qnormc <- qnorm #quantile function

In [24]:
# Prepare data
# ------------------------------------------------------------------------
# Load Fst
fst.HVAxHVC<-fread("timemaHVAxHVC.fst.dsv", header=T, sep="\t")

In [25]:
# Use only one linkage group for this example
fst<-fst.HVAxHVC[grep("^lg01_", fst.HVAxHVC$locus),]$fst

In [26]:
# Remove missing data
fst<-fst[!is.na(fst)]

# Replace too low Fst values by tractable value
fst[fst<1e-4]<-1e-4

# logit transform Fst, we will assume a normal logit distribution 
# for each HMM state
lfst<-log(fst/(1-fst))

# specify nloci, mean and sd (used below)
nloci<-length(lfst)
mean.lfst<-mean(lfst)
sd.lfst<-sd(lfst)
# ------------------------------------------------------------------------

In [27]:
# Fit homogenous HMM of 3 discrete states
# ------------------------------------------------------------------------
# initial m x m transition probability matrix
# disable transitions from 1 to 3 and 3 to 1 (set value to zero) 
# (direct transitions from low to high and high to low differentiation)
Pi<-matrix(c(0.9,0.1,0,
             0.05,0.9,0.05,
             0,0.1,0.9),nrow=3,byrow=T)
Pi
# it is strongly recommended to run the EM algorithm multiple times (hundreds
# or thousands) with different starting matrices to avoid maximum-likelihood
# local optima; e.g. code to produce 100 random matrices:
# random.matrix<-function(x){
#                   x<-runif(3)
#                   matrix(c(x[1],1-x[1],0,
#                            x[2]/2,1-x[2],x[2]/2,
#                            0,x[3],1-x[3]),nrow=3,byrow=T)}
# matrices<-lapply(1:100, random.matrix)

[,1] [,2] [,3]
[1,] 0.90 0.1  0.00
[2,] 0.05 0.9  0.05
[3,] 0.00 0.1  0.90

In [28]:
# initial marginal probability distribution of the m hidden states
delta<-c(0,1,0)

# initial list of parameter values associated with the distribution 
# of observed process
# they must coincide with the parameters of the function use
# we are using a normal and must give 3 means and 3 sds
pm<-list(mean=c(mean.lfst*0.9,mean.lfst,mean.lfst*1.1),
         sd=rep(sd.lfst,3))

In [29]:
# set discrete time HMM object and initial values 
# normc is modified distribution so that variance is fixed
# to genome-wide estimate (sd.lfst)
init<-dthmm(x=lfst,
            Pi=Pi,
            delta=delta,
            distn="normc",
            pm=pm,
            discrete=FALSE)

In [31]:
# estimate parameters of HMM using Baum-Welch EM algorithm
# this may take a while (a few minutes)

# set maximum number of iterations and convergence criterion
ctrl<-bwcontrol(maxiter=1000, tol=1e-6)

# run BaumWelch algorithm
paramHMM<-BaumWelch(init, ctrl)

# compare initial and estimated transition matrices
# initial
Pi
# estimate
paramHMM$Pi

# compare initial and estimated distribution parameters
# (normal means and sds)
# initial
pm
# estimate
paramHMM$pm

In [ ]:
# predict Markov states using Viterbi algorithm
fitHMM<-Viterbi(paramHMM)
# ------------------------------------------------------------------------------

In [ ]:
# Summarize results
# ------------------------------------------------------------------------------
# mean FSTs of each state: high, medium, and low differentiation
# (reverse transform logit FSTs means)
fst.states<-round(1/(1+exp(-1 * paramHMM$pm$mean)),5)
fst.states

# number of loci in each state
nloci.states<-table(fitHMM)
nloci.states

# proportion of loci in each state
proploci.states<-nloci.states/nloci
proploci.states

# number of contiguous regions for each state
nregions.states<-unlist(lapply(1:3,
                    function(x) sum((fitHMM==x)[1:(nloci-1)]-(fitHMM==x)[2:nloci]==1)))
nregions.states

# put everything together in a table
summary.table<-cbind(state=c(1,2,3),
               mean.fst=fst.states,
               nloci=nloci.states,
               proploci=proploci.states,
               nregions=nregions.states)
# show table
summary.table

# save to file
write.table(summary.table, file="fst_hmm_lg01_regions_summary.dsv",
              quote=F, row.names=F, sep="\t")
# ------------------------------------------------------------------------------

In [ ]:
# plot regions using differentiation legend
# ------------------------------------------------------------------------------
par(mar=c(10,3,10,4)+1.5)
colours<-c("red","dark grey","blue")
plot(fst, col=colours[fitHMM], ylim=c(0,0.8),
     xlab="relative position", ylab=expression("F"[ST]), 
     main="regions of differentiation",
     cex=1, cex.axis=1, cex.lab=1, cex.main=1.3, mgp=c(7,3,0))

# add horizontal line to illustrate top 0.1% FST values
abline(h=quantile(fst, prob=0.999,na.rm=T), col="black", lty=3, cex=4, lwd=3)
text(0,quantile(fst, prob=0.999,na.rm=T)+0.025, 
      label=expression("">=" 0.1%"), adj=1, pos=2, cex=1)

# legend
legend("topright",legend=c("high", "medium", "low"), 
       pch=16, col=colours, title="differentiation", bty="n", cex=1)

# ------------------------------------------------------------------------------

In [ ]:
# load the order and size of scaffolds in the draft genome
# ------------------------------------------------------------------------------
lgordscalen<-read.table("data/lg_ord_sca_length.dsv",header=T,sep="\t")
lgordscalen<-lgordscalen[lgordscalen$lg==1,]

loci.pos<-as.numeric(gsub(".*:","",fst.HVAxHVC$locus))
loci.ord<-as.numeric(gsub(".*ord|_scaf.*","",fst.HVAxHVC$locus))
loci.sca<-as.numeric(gsub(".*_scaf|:.*","",fst.HVAxHVC$locus))
# ------------------------------------------------------------------------------

In [ ]:
# high differentiation regions
# ------------------------------------------------------------------------------
lb.locindex<-which(fitHMM[2:nloci]==1 & fitHMM[1:(nloci-1)]!=1)
ub.locindex<-which(fitHMM[1:(nloci-1)]==1 & fitHMM[2:nloci]!=1)
if (length(lb.locindex)<length(ub.locindex)) lb.locindex<-c(1,lb.locindex)
if (length(lb.locindex)>length(ub.locindex)) ub.locindex<-c(ub.locindex,nloci)

hidif.regions<-get.regions(lb.locindex,ub.locindex,loci.ord,loci.pos,lgordscalen)
write.table(hidif.regions, file="fst_hmm_lg01_hidif_regions.dsv", quote=F, sep="\t")
# ------------------------------------------------------------------------------

In [ ]:
# medium differentiation regions
# ------------------------------------------------------------------------------
lb.locindex<-which(fitHMM[2:nloci]==2 & fitHMM[1:(nloci-1)]!=2)
ub.locindex<-which(fitHMM[1:(nloci-1)]==2 & fitHMM[2:nloci]!=2)
if (length(lb.locindex)<length(ub.locindex)) lb.locindex<-c(1,lb.locindex)
if (length(lb.locindex)>length(ub.locindex)) ub.locindex<-c(ub.locindex,nloci)

medif.regions<-get.regions(lb.locindex,ub.locindex,loci.ord,loci.pos,lgordscalen)
write.table(medif.regions, file="fst_hmm_lg01_medif_regions.dsv", quote=F, sep="\t")
# ------------------------------------------------------------------------------

In [ ]:
# low differentiation regions
# ------------------------------------------------------------------------------
lb.locindex<-which(fitHMM[2:nloci]==3 & fitHMM[1:(nloci-1)]!=3)
ub.locindex<-which(fitHMM[1:(nloci-1)]==3 & fitHMM[2:nloci]!=3)
if (length(lb.locindex)<length(ub.locindex)) lb.locindex<-c(1,lb.locindex)
if (length(lb.locindex)>length(ub.locindex)) ub.locindex<-c(ub.locindex,nloci)

lodif.regions<-get.regions(lb.locindex,ub.locindex,loci.ord,loci.pos,lgordscalen)
write.table(lodif.regions, file="fst_hmm_lg01_lodif_regions.dsv", quote=F, sep="\t")
# ------------------------------------------------------------------------------

In [ ]:
# Compare distribution of sizes
# ------------------------------------------------------------------------------
par(mfrow=c(1,3))
par(mar=c(5,5,10,3)+1)
hist(hidif.regions$length/1000,main="high differentiation regions",
        xlim=c(0,300),xlab="size (Kbp)",breaks=50, 
        cex.axis=1, cex.lab=1, cex.main=1)
hist(medif.regions$length/1000,main="medium differentiation regions",
        xlim=c(0,300), xlab="size (Kbp)",breaks=50, cex=4, 
        cex.axis=1, cex.lab=1, cex.main=1)
hist(lodif.regions$length/1000,main="low differentiation regions",
        xlim=c(0,300), xlab="size (Kbp)", breaks=50, cex=4, 
        cex.axis=1, cex.lab=1, cex.main=1)
# ------------------------------------------------------------------------------